In [ ]:
from _2_dm_class import *
n = "2"
protid = "1UAO"
digfmt = str(len(str(n)))
digfmt = '{:0'+digfmt+'d}'
dist_fun = "dihedDist"
fx = "DD"

In [ ]:
######################################################################    
# Calc. DMAP for a trajectory w/o Coarse-graining
def simpleTrajDMap(dist_fun="RMSD", fx="RMSD", stride=100, n=1, digfmt="", nestride=False):
    native_fn = "../../100ns_long_implicit/"+protid+"/"+protid+"-folded.pdb"  
    
    data = UnivData(native_fn)
    
    nc1 = "0"
    nc2 = "35"
    if int(n)==1:
        trj_fn = "../../100ns_long_implicit/"+protid+"/clustering/clustering1/dhd-op_DCDs-500/medoids.dcd" 
       
    else:
        trj_fn = []

        folder_na = "../../100ns_long_implicit/"+protid+"/medoids-500/"
        for i in ["0"]:#range(int(n)):
            trj_fn.append(folder_na+protid+"-round"+i+".dcd")
        
    traj = Traj(trj_fn, native_fn, stride=stride)
    M = Measures(data, traj)
    M.assignFuncs(dist_fun=dist_fun, kernel=M.gaussianKernel)
    M.resetMeasures()

    M.call(data.native, flag="calc")    
    #traj.traj = md.join(traj.traj)
    #traj.nl = np.sum(traj.nl)
    #traj.nl = traj.traj.n_frames

    M.traj = traj
    dmap = DMAPs(data)
    fn = protid+"-"+str(nc1)
    eVal, eVec = dmap.callSimple(traj, n1=M.nc, fn=fn, fx=fx, wFlag=False)
    
    n = 2
    #nc2 = "5-1"
    if nestride:
        #del traj#, M
        oldncl = len(M.nc)
        trj_fn = []

        fnp = "../../"+protid+"/ext/"
        for i in range(1,n+1):
            trj_fn.append(fnp+protid+"-round"+str(i)+".dcd")
        
        stride = nestride
        traj1 = Traj(trj_fn, native_fn, stride=stride)
        #traj1.traj = traj1.traj[1::2]
        #traj1.nl = traj1.traj.n_frames
        M.traj = traj1        
        M.call(data.native, flag="calc")

        traj1.traj = md.join(traj1.traj)
        traj1.nl = np.sum(traj1.nl)
        M.traj = traj1
        eVec = dmap.callSimpleNE(traj, n1=M.nc[:oldncl], n2=M.nc[oldncl:], fn=fn+"-"+str(stride), fx=fx, wFlag=False)
    eVec = dmap.extractDMap()
    
    print2file(fn, fx+'.DMAP', eVec.T, fflag='full',writeflag='w')    
    print2file(fn, fx+'.eVal', eVal.T, fflag='full',writeflag='w')
    M.hpq = M.hpq/max(M.hpq)
    print2file(fn, "colors", np.array([M.rmsd,M.dhd,M.q,M.hpq,M.hisasahp,M.hisasaall,M.nc]).T, fflag='full',writeflag='w')
    
    
    #M.dhd = (-np.array(M.dhd)).tolist()
    if eVec.shape[0]==1:
        pts, col = dmap.selDMapExtremes([M.hisasaall, M.hisasahp], eVec.flatten())
    else:
        #pts, col = dmap.selDMapExtremes(eVec, M.hisasaall)
        pts, col = dmap.selDMapExtremes(eVec, M.hisasahp)
        #pts, col = dmap.selDMapExtremes(eVec, -1*np.array(M.q))
        pts, col = dmap.selDMapExtremes(eVec, M.dhd)
    
        
    dmap.plot(eVec, 
              c={'q':M.q, 'dhd':M.dhd, 'hpq':M.hpq, 'hisasaall':M.hisasaall, 'hisasahp':M.hisasahp, 'nc':M.nc}, 
              cpairs=[['q','dhd'],['hpq','dhd'],['hisasaall','hisasahp']])     
    
    for i in pts:
        print(str(i[0])+"\t"+str(M.q[i[0]])+"\t"+str(M.dhd[i[0]])+"\t"+str(M.rmsd[i[0]]))
    
    print(max(M.q))
    print(min(M.hisasaall), max(M.hisasaall))

    
    M.q = np.array(M.q)
    print(np.where(M.q==max(M.q)))

    
    if len(pts)>0:
        traj.writePDBbyN(pts, "../../PSO-DMA-stretched/"+protid+"/round-"+str(int(nc1)+1)+"/"+protid)
    #pts = np.where(eVec[0]>0.20)
    #pts = np.where(np.array(M.dhd)>0.85)
    #for i in pts:
    #    print(str(i)+"\t"+str(M.q[i]))
    #if len(pts)>0:
    #    traj.writePDBbyN(pts, "../../DESRES-Trajectory_"+protid+"-0-protein/start_from_pdbs"+str(int(nc)+1)+"/orig-higher/"+protid)
    
    del traj, M
    return eVal, eVec
######################################################################    

eVal, eVec = simpleTrajDMap(dist_fun=dist_fun, fx=fx, stride=1, n=1, digfmt=digfmt, nestride=False)

In [ ]:
"""
#eVec_100_25 = simpleTrajDMap(dist_fun=dist_fun, fx=fx, stride=100, n="001", digfmt="", nestride=25)
#eVec_25 = simpleTrajDMap(dist_fun=dist_fun, fx=fx, stride=25, n="001", digfmt="", nestride=False)
scatter(eVec_100_25[1], eVec_25[1])
show()
#from scipy import stats
slope, intercept, r_value, p_value, std_err = stats.linregress(eVec_100_25[4],eVec_25[4])
print("r-squared:", r_value**2)
"""

In [ ]:
######################################################################    
# Calc. DMAP for a single trajectory w/ Coarse-graining
def CGTrajDMap(dist_fun="RMSD", fx="RMSD", neflag=False):
    #global pid 
    pid = 't'
    native_fn = "../../"+protid+"/start_as_beta_sheet/"+protid+"-sheet.pdb"
        
    data = UnivData(native_fn)
    nc1 = "1"
    nc2 = "sqrt"
    # define medoids
    folder_na = "../../"+protid+"/start_as_beta_sheet/clustering/clustering"+nc1+"/"
    t = np.loadtxt(folder_na+"clusters/"+pdbid+nc1+"-"+nc2+".dhd.pam.clusters", dtype=np.uint16)
    #medoids = np.loadtxt(folder_na+"clusters/"+pdbid+nc1+"-"+nc2+".dhd.pam.medoids", dtype=np.uint16)
    
    #folder_na = "../../DESRES-Trajectory_"+pdbid+"-0-protein/"+pdbid+"-0-protein"
    
    nclusts = np.unique(t).shape[0]
    #nclusts = 10
    
    digfmt1 = str(len(str(t.max()-1)))
    digfmt1 = folder_na+"dhd-op_DCDs-"+nc2+"/clust-"+'{:0'+digfmt1+'d}' 
    #digfmt1 = folder_na+"/"+pdbid+"-0-protein-"+'{:0'+digfmt1+'d}' 
    
    
    dmap = DMAPs(data) 
    stride = 5
    fn = pdbid+nc1+"-"+str(stride)+"-d"
    eVal, eVec, M = dmap.callCG(native_fn, digfmt1, dist_fun, nclusts, fn=fn, fx=fx, stride=stride, wFlag=True)   

    if neflag:
        fn = pdbid+nc1+"-"+str(stride)+"-d"
        eVec, M = dmap.callCGNE(native_fn, digfmt1, M, nclusts=[12,170], fn=fn, fx=fx, stride=stride, wFlag=False)
    else:
        M.call([0], flag="NE")
        
    eVec = dmap.extractDMap()

    if eVec.shape[0]==1:
        pts, col = dmap.selDMapExtremes(np.array([M.hisasaall, M.hisasahp]), eVec.flatten())
    else:
        pts, col = dmap.selDMapExtremes(eVec, M.hisasaall)
        pts, col = dmap.selDMapExtremes(eVec, M.hisasahp)
        pts, col = dmap.selDMapExtremes(eVec, M.q)
    #dmap.plot(eVec, 
    #          c={'q':M.q, 'dhd':M.dhd, 'hisasaall':M.hisasaall, 'hisasahp':M.hisasahp, 'selpts':col}, 
    #          cpairs=[['q','dhd'],['hisasaall','hisasahp']])    
    for i in pts:
        print(str(i[0])+"\t"+str(M.q[i[0]]))
    print(max(M.q))
    #M.q = np.array(M.q)
    #print(np.where(M.q==max(M.q)))
    
    #trj_fn = folder_na+"op_DCDs/medoids.dcd"
    #traj = Traj(trj_fn, native_fn)
    #traj.writePDBbyN(pts, "../../../DESRES-Trajectory_"+pdbid+"-0-protein/start_from_pdbs2/orig-DD-CG/"+pdbid)

    
    
    return eVal, eVec
######################################################################    
    
#eVal, eVec = CGTrajDMap(dist_fun=dist_fun, fx=fx, neflag=False)

In [ ]:
from scipy import stats
import copy

######################################################################   
# Calc. DMAP for a trajectory w/o Coarse-graining
def compareTrajDMap(dist_fun="RMSD", fx="RMSD", stride=100, n=1, digfmt="", nestride=False):
    native_fn = "../../"+protid+"/start_as_beta_sheet/"+protid+"-sheet.pdb"
    
    data = UnivData(native_fn)
        
    trj_fn = []
    dmap = []
    eVal = []
    eVec = []
    for i in range(int(n)):
        trj_fn.append("../../"+protid+"/start_as_beta_sheet/"+pdbid+"-"+digfmt.format(i)+".dcd")
        dmap.append(0)
        eVal.append(0)
        eVec.append(0)  
    
    for i in range(int(n)):
        traj = Traj(trj_fn[i], native_fn, stride=stride)
        M = Measures(data, traj)
        M.assignFuncs(dist_fun=dist_fun, kernel=M.gaussianKernel)
        M.resetMeasures()
        M.call(data.native, flag="calc")  
        dmap[i] = DMAPs(data)
        eVal[i], eVec[i] = dmap[i].callSimple(traj, fn="", fx=fx, wFlag=False, verbose=False)
        M.resetMeasures()


    r = []
    for i in range(int(n)):
        #eVec1 = eVec[i]
        r2 = []       
        
        for j in range(int(n)):
            traj = Traj(trj_fn[i], native_fn, stride=stride)
            M = Measures(data, traj)
            M.assignFuncs(dist_fun=dist_fun, kernel=M.gaussianKernel)
            M.resetMeasures()
            M.call(data.native, flag="calc") 
            dmap2 = DMAPs(data)
            eVal1, eVec1 = dmap2.callSimple(traj, fn="", fx=fx, wFlag=False, verbose=False)

            traj1 = Traj(trj_fn[j], native_fn, stride=stride)
            M.traj = traj1
            #M = Measures(data, traj)
            #M.assignFuncs(dist_fun=dist_fun, kernel=M.gaussianKernel)
            #M.resetMeasures()
            M.call(data.native, flag="calc") 
            #dmap2 = copy.deepcopy(dmap1)
            eVec2 = dmap2.callSimpleNE(traj, n1=[1], n2=[2], fn="", fx=fx, wFlag=False, verbose=False)
            scatter(eVec[j][1],eVec2[1][eVec2.shape[0]:])
            show()
            slope, intercept, r_value, p_value, std_err = stats.linregress(eVec[j][1],eVec2[1][eVec2.shape[0]:])
            r2.append((r_value**2).round(3))
            M.resetMeasures()

        r.append((r2))
    #r = np.array(r)
    print(r)
    
######################################################################    
    
#compareTrajDMap(dist_fun=dist_fun, fx=fx, stride=1, n=n, digfmt=digfmt, nestride=False)